# FACEBOOK PLACES

In [ ]:
rm(list=ls())

*W jupyterze dobrze jest skonfrontować z tym co ma się na swoim komputerze, bo później mogą nas spotkać niespodzianki*

In [ ]:
R.Version()$version

## O projekcie

:)

## Wstęp

Facebook posiada opcję wyszukiwania miejsc, chyba znacznie mniej popularną od googlowej, ale przydatną, m.in. ze względu na liczbę informacji o lokalizacji pochodzących z dość prężnie działającej strony SM:    
https://www.facebook.com/search/str/club+warszawa/keywords_search

Korzystając z API możemy wyszukiwać tych miejsc z poziomu Ra.   
W pierwszej kolejności trzeba ustalić listę przydatnych pakietów:

In [83]:
pakiety <- c("httr", "jsonlite", "lubridate",'RCurl','magrittr');
lapply(pakiety, library, character.only = TRUE);

[[1]]
 [1] "ggplot2"   "magrittr"  "RCurl"     "bitops"    "lubridate" "jsonlite" 
 [7] "httr"      "stats"     "graphics"  "grDevices" "utils"     "datasets" 
[13] "methods"   "base"     

[[2]]
 [1] "ggplot2"   "magrittr"  "RCurl"     "bitops"    "lubridate" "jsonlite" 
 [7] "httr"      "stats"     "graphics"  "grDevices" "utils"     "datasets" 
[13] "methods"   "base"     

[[3]]
 [1] "ggplot2"   "magrittr"  "RCurl"     "bitops"    "lubridate" "jsonlite" 
 [7] "httr"      "stats"     "graphics"  "grDevices" "utils"     "datasets" 
[13] "methods"   "base"     

[[4]]
 [1] "ggplot2"   "magrittr"  "RCurl"     "bitops"    "lubridate" "jsonlite" 
 [7] "httr"      "stats"     "graphics"  "grDevices" "utils"     "datasets" 
[13] "methods"   "base"     

[[5]]
 [1] "ggplot2"   "magrittr"  "RCurl"     "bitops"    "lubridate" "jsonlite" 
 [7] "httr"      "stats"     "graphics"  "grDevices" "utils"     "datasets" 
[13] "methods"   "base"

W drugiej kolejności musimy zdobyć token, który pozwoli nam korzystać z FB for developers. Generujemy go sobie pod tym linkiem (musimy mieć konto na fejsie): 
https://developers.facebook.com/tools/explorer/

Jest to tymczasowy token, który jest ważny jakąś plus/minus godzinę. Można chyba wygenerować stały, ale to wiąże się z:
* potancjalnymi kosztami
* koniecznością przyswojenia sobie dokumentacji, która nie wydawała mi się jakoś bardzo przystępna
* podaniem numery własnej karty kredytowej :<<<<



In [1]:
tmp_token = 'TOKEN'

## Zbieranie danych

Mamy kilka możliwości korzystania z FB Places API:
* wyszukiwanie miejsc we wskazanej okolicy na podstawie współrzędnych (**Places Search**)
* pozyskiwanie szczegółów nt miejsc na podstawie FB id (**Places details**, najpierw jednak trzeba mieć **id lokalizacji**)
* i inne, z których nie korzystałam

Dla ciekawych tutaj jest sporo informacji, nie do końca wszystko działa tak jak napisano, ale przy odrobinie domyśloności damy radę:
https://developers.facebook.com/docs/places/web

W naszym przypadku najpierw zbieramy miejsca z okolicy (dzięki czemu zyskamy ID), a następnie po ID znajdziemy szczegóły lokalizacji.

### Procedura

Wyszukanie lokali    
1. Ustalić czego szukamy   
2. Wygenerować token https://developers.facebook.com/tools/explorer/
3. Wygenerować URL z odpowiednimi parapetrami ( w tym z naszym tokenem )    
4. Ściągnąć pierwszą 'transzę'    
5. Pobrać link do kolejnych 'podstron'
6. Zapisać wyniki    

Pobranie dodatkowych danych    

7. Ściągnąć szczegóły wg ID zdobytego w etapach 1-6


### Places search

In [84]:
#wspolrzedne: &center= 
#odleglosc wyszukiwania wokół wspolrzednych: &distance=
#lista pól do pobrania: &fields=

url_p1 <- 'https://graph.facebook.com/v2.11/search?type=place&'

# slowo klucz, mozemy szukac tez cafe, pub, bar, cinema etc
q = 'q=club' # gdy podamy takie slowo klucz bedzie szukal obiektow o kategorii 'club' i miejsc ze slowem 'club' w nazwie
# gdy wpiszemy 'klub' dostaniemy tylko miejsca z nazwa 'klub', bo FB nie rozpoznaje tego jako swojej kategorii

url_p2 <- '&center=52.2297700,21.0117800&distance=15000&fields=name,overall_star_rating,category_list&access_token='

# to ma zwiazek z pojedynczym limitem pobran
url_p3 <- '&limit=1000&before=1&after=10'
  
URL <- paste0(url_p1,q,url_p2,tmp_token,url_p3) # tu sklejam URL, jest podzielony, bo zmieniałam współrzędne, q etc

lista_lokali_1 <- getURL(URL) %>% fromJSON()
lista_lokali_1 %>% head

name,category_list,id,overall_star_rating
Club Wesele Żurawia róg Parkingowej,"191478144212980 , Dance & Night Club",1182681298459274,NA
ClubBoho22,"273819889375819 , 218693881483234 , 173883042668223 , Restaurant , Pub , Performance Art Theatre",303660730090101,4.3
Sultan Club,"223801560977980 , 133436743388217 , 186004504854452 , Hookah Lounge , Arts & Entertainment , Country Club / Clubhouse",251423338388345,4.1
Vis-a-vis Club&Bar,"150805268316547 , 192108214153222 , 138756772860047 , Polish Restaurant , Breakfast & Brunch Restaurant, Creperie",174102442684643,4.0
GLAM CLUB,"191478144212980 , 187623434605561 , 186004504854452 , Dance & Night Club , Adult Entertainment Service, Country Club / Clubhouse",113850325306747,NA
Libido Night Club,"191478144212980 , Dance & Night Club",213510888800998,3.8
Club Stereo,"191478144212980 , 218693881483234 , Dance & Night Club, Pub",198731159280,4.4
BANK CLUB,"191478144212980 , 110290705711626 , Dance & Night Club, Bar",125778890814493,NA
N58 Club,"191478144212980 , 2612 , Dance & Night Club, Community",1856284488020975,4.8
Rotor Club,"176831012360626 , 174177802634376 , Professional Service, Hair Salon",118597681528808,4.8


Lista lokali jest strasznym bałaganem, więc przepisujemy sobie tylko to co jest nam potrzebne, czyli $data

In [85]:
lista_lokali_1$data %>% head

name,category_list,id,overall_star_rating
Club Wesele Żurawia róg Parkingowej,"191478144212980 , Dance & Night Club",1182681298459274,NA
ClubBoho22,"273819889375819 , 218693881483234 , 173883042668223 , Restaurant , Pub , Performance Art Theatre",303660730090101,4.3
Sultan Club,"223801560977980 , 133436743388217 , 186004504854452 , Hookah Lounge , Arts & Entertainment , Country Club / Clubhouse",251423338388345,4.1
Vis-a-vis Club&Bar,"150805268316547 , 192108214153222 , 138756772860047 , Polish Restaurant , Breakfast & Brunch Restaurant, Creperie",174102442684643,4.0
GLAM CLUB,"191478144212980 , 187623434605561 , 186004504854452 , Dance & Night Club , Adult Entertainment Service, Country Club / Clubhouse",113850325306747,NA
Libido Night Club,"191478144212980 , Dance & Night Club",213510888800998,3.8


API nie pozwola nam na pobranie wszystkich warszawskich klubów za jednym razem, listy miejsc są rozpisane na kolejnych "stronach" (w widoku dla użytkownika objawia się to potrzebą "scrollowania"). Stąd z naszego już pobranego obiektu (pierwszej serii miejsc) musimy wygrzebać link do kolejnej strony:

In [86]:
lista_lokali_1$paging['next'] %>% print

$`next`
[1] "https://graph.facebook.com/v2.11/search?access_token=EAACEdEose0cBAONx8i1QlKZCQSRAgvcpSXuGKoYEqK86BWnGXou0OJpaW8AF1KuQm2B3rhKOcxOrHc1TXR89RAAySPkV6DZCYm5i7Q7inArq8tBGLAT4NLccqoct8eGweGKJoZAC2YvZAVQaKU4hbJNZAIa5ppNEgmCFm90VG7jZCVN1OaGfp8MembE3HVlZAaJdIFDQGISXwZDZD&fields=name%2Coverall_star_rating%2Ccategory_list&q=club&type=place&center=52.2297700%2C21.0117800&distance=15000&limit=100&after=MTAw"



Zerknijmy ile mamy obiektów przy pierwszym podejściu:

In [87]:
lista_lokali_1$data$name %>% length # max

[1] 100

Część z tej pętli jest zakomentowana, bo w jupyterze dostaję trochę inne klasy obiektów niż w R, więc nie działa tak jak powinno, a nie warto marnować czasu (skutek różnic w wersji Ra w Jupyterze i na komputerze roboczym):

In [88]:
for (i in 2:4){
    url <- paste0('lista_lokali_',i-1) %>% get # pobieram poprzednio (i-1) pobrany obiekt z listą lokali
    url <- as.character(url$paging['next']) # zapisuje ulr do pobrania kolejnej

    # zapisuje i-ty obiekt, z którego skorzysta kolejny przebieg pętli
    assign(paste0('lista_lokali_',i),getURL(url) %>% fromJSON()) 

    # zapis danych w tmp
    #tmp <- paste0('lista_lokali_',i) %>% get 
    #tmp <- tmp$data
    
    # appendowanie zdobytych list
    #lista_lokali <- append(lista_lokali,tmp)
}

Widać, że mamy trochę duplikatów:

In [89]:
lista_lokali <- rbind(lista_lokali_1$data,lista_lokali_2$data,lista_lokali_3$data,lista_lokali_4$data)

lista_lokali %>% dim
lista_lokali$id %>% unique %>% length

[1] 380   4

[1] 357

In [90]:
kategorie <- lista_lokali$category_list
lista_lokali$ile_kat <- lapply(1:nrow(lista_lokali),function(x) kategorie[[x]]$name %>% length) %>% unlist
#head(lista_lokali)

In [91]:
lista_lokali[,paste0('kat_',1:3)] <- NA

In [92]:
kategorie[1:3] %>% head()

id,name
191478144212980,Dance & Night Club
id,name
273819889375819,Restaurant
218693881483234,Pub
173883042668223,Performance Art Theatre
id,name
223801560977980,Hookah Lounge
133436743388217,Arts & Entertainment
186004504854452,Country Club / Clubhouse


In [93]:
lista_lokali %>% head

name,category_list,id,overall_star_rating,ile_kat,kat_1,kat_2,kat_3
Club Wesele Żurawia róg Parkingowej,"191478144212980 , Dance & Night Club",1182681298459274,NA,1,NA,NA,NA
ClubBoho22,"273819889375819 , 218693881483234 , 173883042668223 , Restaurant , Pub , Performance Art Theatre",303660730090101,4.3,3,NA,NA,NA
Sultan Club,"223801560977980 , 133436743388217 , 186004504854452 , Hookah Lounge , Arts & Entertainment , Country Club / Clubhouse",251423338388345,4.1,3,NA,NA,NA
Vis-a-vis Club&Bar,"150805268316547 , 192108214153222 , 138756772860047 , Polish Restaurant , Breakfast & Brunch Restaurant, Creperie",174102442684643,4.0,3,NA,NA,NA
GLAM CLUB,"191478144212980 , 187623434605561 , 186004504854452 , Dance & Night Club , Adult Entertainment Service, Country Club / Clubhouse",113850325306747,NA,3,NA,NA,NA
Libido Night Club,"191478144212980 , Dance & Night Club",213510888800998,3.8,1,NA,NA,NA


In [94]:
for (i in 1:nrow(lista_lokali)) { # to jest liczba wierszy
  for (j in 1:lista_lokali[i,]$ile_kat) { # liczba kategorii jakie przypisal fejsik danej lokalizacji
       lista_lokali[i,5+j] <- kategorie[[i]]$name[[j]]
  }
}

lista_lokali$category_list <- NULL

In [95]:
lista_lokali %>% head(10)

name,id,overall_star_rating,ile_kat,kat_1,kat_2,kat_3
Club Wesele Żurawia róg Parkingowej,1182681298459274,NA,1,Dance & Night Club,NA,NA
ClubBoho22,303660730090101,4.3,3,Restaurant,Pub,Performance Art Theatre
Sultan Club,251423338388345,4.1,3,Hookah Lounge,Arts & Entertainment,Country Club / Clubhouse
Vis-a-vis Club&Bar,174102442684643,4.0,3,Polish Restaurant,Breakfast & Brunch Restaurant,Creperie
GLAM CLUB,113850325306747,NA,3,Dance & Night Club,Adult Entertainment Service,Country Club / Clubhouse
Libido Night Club,213510888800998,3.8,1,Dance & Night Club,NA,NA
Club Stereo,198731159280,4.4,2,Dance & Night Club,Pub,NA
BANK CLUB,125778890814493,NA,2,Dance & Night Club,Bar,NA
N58 Club,1856284488020975,4.8,2,Dance & Night Club,Community,NA
Rotor Club,118597681528808,4.8,2,Professional Service,Hair Salon,NA


Lista kategorii jest długa i ciekawa, dla nas ciekawe były raczej lokalizacje zawierające:
* Dance & Night Club
* Pub
* Diver Bar    

Odrzucałam wszystkie sports cluby etc.

In [98]:
c(lista_lokali$kat_1, lista_lokali$kat_2, lista_lokali$kat_3) %>% unique 

[1] "Dance & Night Club"                   
  [2] "Restaurant"                           
  [3] "Hookah Lounge"                        
  [4] "Polish Restaurant"                    
  [5] "Professional Service"                 
  [6] "Bar & Grill"                          
  [7] "Gym/Physical Fitness Center"          
  [8] "Pub"                                  
  [9] "Martial Arts School"                  
 [10] "Apartment & Condo Building"           
 [11] "Tanning Salon"                        
 [12] "Community"                            
 [13] "Jazz & Blues Club"                    
 [14] "Personal Coach"                       
 [15] "Karaoke"                              
 [16] "Local Business"                       
 [17] "Hotel"                                
 [18] "College & University"                 
 [19] "Arts & Entertainment"                 
 [20] "Performance & Event Venue"            
 [21] "Dance Studio"                         
 [22] "Business Center"                      
 [23] "Adult Entertainment Service"          
 [24] "Business Service"                     
 [25] "Barber Shop"                          
 [26] "Hair Salon"                           
 [27] "Sports Club"                          
 [28] "Baby & Children's Clothing Store"     
 [29] "Party Entertainment Service"          
 [30] "Organization"                         
 [31] "Stadium, Arena & Sports Venue"        
 [32] "Recreation Center"                    
 [33] "Nonprofit Organization"               
 [34] "Pool & Billiard Hall"                 
 [35] "Big Box Retailer"                     
 [36] "Country Club / Clubhouse"             
 [37] "Amateur Sports Team"                  
 [38] "Health/Beauty"                        
 [39] "Tour Agency"                          
 [40] "Automotive Storage Facility"          
 [41] "Nail Salon"                           
 [42] "Sports & Recreation"                  
 [43] "Event Planner"                        
 [44] "Airline Industry Service"             
 [45] "Bar"                                  
 [46] "Foodservice Distributor"              
 [47] "Education"                            
 [48] "Tennis Court"                         
 [49] "Just For Fun"                         
 [50] "Golf Course & Country Club"           
 [51] "Sushi Restaurant"                     
 [52] "General Dentist"                      
 [53] "Sports"                               
 [54] "Non-Governmental Organization (NGO)"  
 [55] "Food & Beverage Company"              
 [56] "Pet Sitter"                           
 [57] "Travel Agency"                        
 [58] "Travel Company"                       
 [59] "Dive Bar"                             
 [60] "Motor Vehicle Company"                
 [61] "Media/News Company"                   
 [62] "Bartending Service"                   
 [63] "Blogger"                              
 [64] "Public Figure"                        
 [65] "Cafe"                                 
 [66] "Social Club"                          
 [67] "Travel Service"                       
 [68] "Sports & Fitness Instruction"         
 [69] "Education Website"                    
 [70] "Art"                                  
 [71] "Movie/Television Studio"              
 [72] "Breakfast & Brunch Restaurant"        
 [73] "Internet Company"                     
 [74] "Beauty Salon"                         
 [75] "Commercial & Industrial"              
 [76] "Language School"                      
 [77] "Sports Team"                          
 [78] "Finance Company"                      
 [79] "Alternative & Holistic Health Service"
 [80] "Charity Organization"                 
 [81] "School"                               
 [82] "Indian Restaurant"                    
 [83] "Burger Restaurant"                    
 [84] "Preschool"                            
 [85] "Community Organization"               
 [86] "Private Members Club"                 
 [87] "Airport"                              


### Places details

Mamy już listę adresów wraz z ID, więc teraz możemy sobie dobrać do naszych lokalizacji szczegóły, których potrzebujemy (no i FB nam je udostępnia).

In [99]:
get_places_detailes <- function(id, token){
  
    www <- paste0('https://graph.facebook.com/v2.11/',
                id,'?fields=hours,checkins,engagement,is_verified,location,price_range,rating_count,website,overall_star_rating')
    
    www <- paste0(www,'&access_token=',token)

    result <- getURL(www) %>% fromJSON()  
}


extract_var <- function(var){
  a <- lapply(1:dlugosc, function(z) ifelse(results[[z]][var] == 'NULL', NA, results[[z]][var])) %>% unlist()
}


To chwilę trwa, więc nie będziemy szukać dla wszystkich lokalizacji, oszczędzimy sobie też uwagi Marka Z.

In [100]:
results <- lapply(lista_lokali[1:80,]$id, get_places_detailes, token = tmp_token)

In [101]:
head(results,2)

[[1]]
[[1]]$hours
[[1]]$hours$fri_1_open
[1] "21:00"

[[1]]$hours$fri_1_close
[1] "06:00"

[[1]]$hours$sat_1_open
[1] "21:00"

[[1]]$hours$sat_1_close
[1] "06:00"


[[1]]$checkins
[1] 6433

[[1]]$engagement
[[1]]$engagement$count
[1] 22213

[[1]]$engagement$social_sentence
[1] "22K people like this."


[[1]]$is_verified
[1] FALSE

[[1]]$location
[[1]]$location$city
[1] "Warsaw"

[[1]]$location$country
[1] "Poland"

[[1]]$location$latitude
[1] 52.22828

[[1]]$location$longitude
[1] 21.01378

[[1]]$location$street
[1] "ul. Żurawia 32/34"

[[1]]$location$zip
[1] "00-515"


[[1]]$price_range
[1] "$"

[[1]]$rating_count
[1] 0

[[1]]$website
[1] "http://www.clubwesele.pl"

[[1]]$id
[1] "1182681298459274"


[[2]]
[[2]]$hours
[[2]]$hours$mon_1_open
[1] "16:00"

[[2]]$hours$mon_1_close
[1] "01:00"

[[2]]$hours$tue_1_open
[1] "16:00"

[[2]]$hours$tue_1_close
[1] "01:00"

[[2]]$hours$wed_1_open
[1] "16:00"

[[2]]$hours$wed_1_close
[1] "01:00"

[[2]]$hours$thu_1_open
[1] "16:00"

[[2]]$hours$thu_1_close
[1] "01:00"

[[2]]$hours$fri_1_open
[1] "16:00"

[[2]]$hours$fri_1_close
[1] "02:00"

[[2]]$hours$sat_1_open
[1] "16:00"

[[2]]$hours$sat_1_close
[1] "02:00"

[[2]]$hours$sun_1_open
[1] "16:00"

[[2]]$hours$sun_1_close
[1] "01:00"


[[2]]$checkins
[1] 247

[[2]]$engagement
[[2]]$engagement$count
[1] 271

[[2]]$engagement$social_sentence
[1] "271 people like this."


[[2]]$is_verified
[1] FALSE

[[2]]$location
[[2]]$location$city
[1] "Warsaw"

[[2]]$location$country
[1] "Poland"

[[2]]$location$latitude
[1] 52.22856

[[2]]$location$located_in
[1] "156596137727090"

[[2]]$location$longitude
[1] 21.01553

[[2]]$location$street
[1] "Żurawia 22"

[[2]]$location$zip
[1] "00-515"


[[2]]$price_range
[1] "$$"

[[2]]$rating_count
[1] 22

[[2]]$overall_star_rating
[1] 4.3

[[2]]$id
[1] "303660730090101"

In [102]:
for (i in 1:length(results)){
      results[[i]]$location$city <- ifelse(is.logical(results[[i]]$location$city)|is.null(results[[i]]$location$city), 
                                                                                          NA, results[[i]]$location$city)
}

In [103]:
df <- lista_lokali[1:80,]
dlugosc <- 80

df <- transform(df,
                checkins = extract_var('checkins'),
                likes_count = lapply(1:dlugosc, function(z) results[[z]]$engagement$count) %>% unlist(),
                verification = extract_var('is_verified'),
                city = lapply(1:dlugosc, function(z) results[[z]]$location$city) %>% unlist(),
                latitude = lapply(1:dlugosc, function(z) results[[z]]$location$latitude) %>% unlist(),
                longitude = lapply(1:dlugosc, function(z) results[[z]]$location$longitude) %>% unlist(),
                rating_count = extract_var('rating_count'),
                price_range = extract_var('price_range'),
                stars = extract_var('overall_star_rating'),
                adres = lapply(1:nrow(df), function(z) ifelse(is.null(results[[z]]$location$street),
                                                          NA,results[[z]]$location$street) %>% unlist()) %>% as.character())

In [104]:
df %>% head

name,id,overall_star_rating,ile_kat,kat_1,kat_2,kat_3,checkins,likes_count,verification,city,latitude,longitude,rating_count,price_range,stars,adres
Club Wesele Żurawia róg Parkingowej,1182681298459274,NA,1,Dance & Night Club,NA,NA,6433,22213,FALSE,Warsaw,52.22828,21.01378,0,$,NA,ul. Żurawia 32/34
ClubBoho22,303660730090101,4.3,3,Restaurant,Pub,Performance Art Theatre,247,271,FALSE,Warsaw,52.22856,21.01553,22,$$,4.3,Żurawia 22
Sultan Club,251423338388345,4.1,3,Hookah Lounge,Arts & Entertainment,Country Club / Clubhouse,3641,2007,FALSE,Warsaw,52.23166,21.01190,202,$$,4.1,Chmielna 35
Vis-a-vis Club&Bar,174102442684643,4.0,3,Polish Restaurant,Breakfast & Brunch Restaurant,Creperie,240,758,FALSE,Srodmiescie,52.22850,21.01047,40,$,4.0,Nowogrodzka 40
GLAM CLUB,113850325306747,NA,3,Dance & Night Club,Adult Entertainment Service,Country Club / Clubhouse,5806,13288,FALSE,Warsaw,52.22856,21.01553,0,$,NA,Żurawia 22
Libido Night Club,213510888800998,3.8,1,Dance & Night Club,NA,NA,262,3909,FALSE,Srodmiescie,52.23766,21.00994,5,NA,3.8,Kredytowa 9


## Obserwacje

Warto pamiętać, że:
* token działa krótko i ma limit (raz się tylko o niego rozbiłam, jeden ctrl + enter za dużo...)
* nie za każdym przepuszczeniem dostaniemy taką samą liczbę klubów
* marne szanse, że dostaniemy je w tej samej kolejności...    

W związku z tym nie warto nic wpisywać 'na twardo' !

* ID FB jest bardzo długie i warto je przechowywać jako tekst z literką/znakiem na początku, bo excel nam zaokrągli ID zamieniając np. ostatnie 4 cyfry na 0...

In [105]:
lista_lokali$id %>% head

[1] "1182681298459274" "303660730090101"  "251423338388345"  "174102442684643" 
[5] "113850325306747"  "213510888800998"

:)

# KONIEC